I want to be able to analyse a circuit and insert gates into it depending on what I find. What is the best way to do this in Qiskit?

Here's a simple problem to illustrate.

Bobe gave Alice the job of making a quantum circuit in Qiskit for a Bell pair. Specifically, he wants the state 

$$\frac{|01\rangle + |10\rangle)}{\sqrt{2}}.$$

Here's what she does.

In [1]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, Aer, execute, compile

q = QuantumRegister(2,'qubit')
c = ClassicalRegister(2,'bit')
qc = QuantumCircuit(q, c)

qc.x(q[1])
qc.cx(q[0], q[1])
qc.measure(q, c)

She delivers the finished circuit to Bob, who then runs it.

In [2]:
backend = Aer.get_backend('qasm_simulator')
job_sim = execute(qc, backend)
sim_result = job_sim.result()

print(sim_result.get_counts(qc))

{'10': 1024}


That's not a Bell pair! Something is not right.

To see what went wrong, Bob inspects the data attribute of the circuit.

In [3]:
qc.data

There's no Hadamard! Alice forgot the Hadamard!

To insert it himself, Bob first needs to work out which qubit was the control. He does that by accessing the second element of the above list (the CNOT gate), and then finding the first qubit listed under the `qargs` attribute of the gate.

In [4]:
print(qc.data[1].qargs[0])
qreg, index = qc.data[1].qargs[0]

(QuantumRegister(2, 'qubit'), 0)


Now he just needs to insert the Hadamard acting on this qubit into the circuit.

**What is the best method to do this?**

Bob tries the following hack. First, make a new circuit using the required register.

In [5]:
qc_temp = QuantumCircuit(qreg)

Then create the required gate on the required qubit with this.

In [6]:
h = qc_temp.h( qreg[ index ] )

Now he can insert this into the list of gates as expressed by `qc.data`.

In [7]:
qc.data.insert(1,h)

In [8]:
print(qc)

                 ┌───┐        ┌─┐
qubit_0: |0>─────┤ H ├──■─────┤M├
            ┌───┐└───┘┌─┴─┐┌─┐└╥┘
qubit_1: |0>┤ X ├─────┤ X ├┤M├─╫─
            └───┘     └───┘└╥┘ ║ 
   bit_0: 0 ════════════════╬══╩═
                            ║    
   bit_1: 0 ════════════════╩════
                                 


It works, but creating a circuit just to create a gate seems like an overly complex way to do something simple.